<a href="https://colab.research.google.com/github/kislay960/Data-Science-Practice/blob/main/Emoji_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
#install the emoji package to display different emojis
!pip install emoji

In [180]:
import emoji

In [181]:
#creating a dictionary for the emojis we will use
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [182]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [183]:
#importing libraries required
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [184]:
#loading the dataset
train = pd.read_csv('https://raw.githubusercontent.com/kislay960/Data-Science-Practice/main/train_emoji.csv',header=None)
test = pd.read_csv('https://raw.githubusercontent.com/kislay960/Data-Science-Practice/main/test_emoji.csv',header=None)

In [185]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [186]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [187]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

In [188]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(132,)
(56,)
(132,)
(56,)


In [189]:
# Splitting the train data from sentences to words
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

# Splitting the test data from sentences to words
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()
    
# Converting labels into categorical form
Y_train = np_utils.to_categorical(Y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [190]:
# Creating  embeddings dictionary with key = word and value = list of words in glove vector
embeddings_index = {}

f = open('/content/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [191]:
# Filling the embedding matrix
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]

In [192]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [231]:
# A simple RNN and LSTM network to classify the emoji class from an input Sentence

model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_14 (SimpleRNN)    (None, 10, 64)            7360      
_________________________________________________________________
dropout_32 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_33 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 5)                 325       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
Total params: 40,709
Trainable params: 40,709
Non-trainable params: 0
_________________________________________________

In [232]:
# Setting Loss and Optimiser
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [233]:
# Training the model and Setting hyperparameters
hist = model.fit(embedding_matrix_train,Y_train,epochs=100, batch_size=64,shuffle=True)

Epoch 1/100
3/3 [==============================] - 0s 7ms/step - loss: 1.6332 - accuracy: 0.1591
Epoch 2/100
3/3 [==============================] - 0s 7ms/step - loss: 1.5774 - accuracy: 0.2803
Epoch 3/100
3/3 [==============================] - 0s 8ms/step - loss: 1.5823 - accuracy: 0.2500
Epoch 4/100
3/3 [==============================] - 0s 7ms/step - loss: 1.5237 - accuracy: 0.3106
Epoch 5/100
3/3 [==============================] - 0s 7ms/step - loss: 1.5079 - accuracy: 0.3333
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 1.4815 - accuracy: 0.3485
Epoch 7/100
3/3 [==============================] - 0s 8ms/step - loss: 1.4869 - accuracy: 0.2879
Epoch 8/100
3/3 [==============================] - 0s 7ms/step - loss: 1.4600 - accuracy: 0.3939
Epoch 9/100
3/3 [==============================] - 0s 7ms/step - loss: 1.4678 - accuracy: 0.3561
Epoch 10/100
3/3 [==============================] - 0s 7ms/step - loss: 1.4456 - accuracy: 0.3409
Epoch 11/100
3/3 [===========

In [235]:
#predicting values for test data
pred = model.predict_classes(embedding_matrix_test)

In [236]:
pred

array([4, 3, 2, 0, 2, 0, 1, 0, 4, 2, 1, 2, 0, 3, 1, 3, 2, 0, 1, 4, 0, 0,
       4, 0, 3, 1, 2, 0, 4, 4, 0, 1, 3, 2, 0, 3, 2, 4, 4, 2, 1, 0, 0, 1,
       2, 0, 2, 2, 3, 1, 3, 0, 3, 2, 2, 0])

In [237]:
# Calculating the accuracy of the algorithm
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.6607142857142857

In [238]:
for i in range(30):
    print(' '.join(X_test[i]))
    print('Predicted emoji : ' + emoji.emojize(emoji_dictionary[str(pred[i])]) + '\n')

I want to eat
Predicted emoji : 🍴

he did not answer
Predicted emoji : 😓

he got a raise
Predicted emoji : 😁

she got me a present
Predicted emoji : ❤️

ha ha ha it was so funny
Predicted emoji : 😁

he is a good friend
Predicted emoji : ❤️

I am upset
Predicted emoji : ⚾

We had such a lovely dinner tonight
Predicted emoji : ❤️

where is the food
Predicted emoji : 🍴

Stop making this joke ha ha ha
Predicted emoji : 😁

where is the ball
Predicted emoji : ⚾

work is hard
Predicted emoji : 😁

This girl is messing with me
Predicted emoji : ❤️

are you serious ha ha
Predicted emoji : 😓

Let us go play baseball
Predicted emoji : ⚾

This stupid grader is not working
Predicted emoji : 😓

work is horrible
Predicted emoji : 😁

Congratulation for having a baby
Predicted emoji : ❤️

stop messing around
Predicted emoji : ⚾

any suggestions for dinner
Predicted emoji : 🍴

I love taking breaks
Predicted emoji : ❤️

you brighten my day
Predicted emoji : ❤️

I boiled rice
Predicted emoji : 🍴

she is a 